# Glassnode - compressed onchain data

### Import

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path 
import datatable as dt

from keras.layers import Dense
from keras import Input, Model

from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


### Get features

In [2]:
filepath = Path('../../data/csv/glassnode_onchain_data.csv')

In [3]:
df = dt.fread(filepath).to_pandas() 
df = df.set_index('t')
df.index.names = ['date']
df.head(3)

,addresses/count,addresses/sending_count,addresses/receiving_count,addresses/active_count,addresses/new_non_zero_count,transactions/count,transactions/rate,transactions/transfers_volume_sum,transactions/transfers_volume_mean,transactions/transfers_volume_median,...,fees/volume_mean,fees/volume_median,blockchain/utxo_created_count,blockchain/utxo_spent_count,blockchain/utxo_created_value_sum,blockchain/utxo_spent_value_sum,blockchain/utxo_created_value_mean,blockchain/utxo_spent_value_mean,blockchain/utxo_created_value_median,blockchain/utxo_spent_value_median
date,,,,,,,,,,,,,,,,,,,,,
2009-01-03,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,50.0,0.0,50.0,0.0,50.0,0.0
2009-01-04,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-05,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Compute signals

#### 1. Prepare data for model

In [4]:
df_train = df[(df.index >= '2013-01-01')&(df.index < '2019-01-01')]
df_val = df[(df.index >= '2019-01-01')&(df.index < '2020-01-01')]
df_test = df[(df.index >= '2020-01-01')]

In [5]:
df_train.shape, df_val.shape, df_test.shape

((2191, 21), (365, 21), (826, 21))

#### 2. Scale data

In [6]:
scaler = StandardScaler().fit(df_train)

X_train_scaled = scaler.transform(df_train)
X_val_scaled = scaler.transform(df_val)
X_test_scaled = scaler.transform(df_test)

#### 3. Build autoencoder model

In [7]:
n = df_train.shape[1]

In [8]:
encoding_dim = [15, 10, 5] #[n//(2*k) for k in range(1,4)]

input_img = Input(shape=(n,))

# encodeding
encodeding_1 = Dense(encoding_dim[0], activation='relu')(input_img)
encodeding_2 = Dense(encoding_dim[1], activation='relu')(encodeding_1)
encoded = Dense(encoding_dim[2], activation='relu')(encodeding_2)

# decodeding
decodeding_1 = Dense(encoding_dim[0], activation='sigmoid')(encoded)
decodeding_2 = Dense(encoding_dim[1], activation='sigmoid')(decodeding_1)
decoded = Dense(n, activation='sigmoid')(decodeding_2)

# Autoencoder - model
autoencoder = Model(input_img, decoded)

# Encoder - model
encoder = Model(input_img, encoded)

# Dencoder - model
encoded_input = Input(shape=(encoding_dim[2],))
decoder_1 = autoencoder.layers[-3](encoded_input)
decoder_2 = autoencoder.layers[-2](decoder_1)
decoder_3 = autoencoder.layers[-1](decoder_2)
decoder = Model(encoded_input, decoder_3)

In [9]:
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 21)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 15)                330       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                160       
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_4 (Dense)              (None, 15)                90        
_________________________________________________________________
dense_5 (Dense)              (None, 10)                160       
_________________________________________________________________
dense_6 (Dense)              (None, 21)                231 

#### 4. Train model

In [10]:
autoencoder.compile(optimizer='adam', loss='mean_absolute_error')
# binary_crossentropy
# mean_squared_error
# mean_absolute_error

In [11]:
autoencoder.fit(X_train_scaled, X_train_scaled,
                epochs=300,
                batch_size=10,
                validation_data=(X_val_scaled, X_val_scaled))

Train on 2191 samples, validate on 365 samples
Epoch 1/300
2191/2191 [==============================] - 0s 172us/step - loss: 0.8152 - val_loss: 1.0095
Epoch 2/300
2191/2191 [==============================] - 0s 101us/step - loss: 0.7228 - val_loss: 0.9078
Epoch 3/300
2191/2191 [==============================] - 0s 102us/step - loss: 0.6627 - val_loss: 0.8076
Epoch 4/300
2191/2191 [==============================] - 0s 101us/step - loss: 0.6284 - val_loss: 0.7489
Epoch 5/300
2191/2191 [==============================] - 0s 101us/step - loss: 0.6114 - val_loss: 0.7230
Epoch 6/300
2191/2191 [==============================] - 0s 102us/step - loss: 0.6023 - val_loss: 0.7124
Epoch 7/300
2191/2191 [==============================] - 0s 101us/step - loss: 0.5958 - val_loss: 0.7078
Epoch 8/300
2191/2191 [==============================] - 0s 101us/step - loss: 0.5909 - val_loss: 0.7125
Epoch 9/300
2191/2191 [==============================] - 0s 102us/step - loss: 0.5865 - val_loss: 0.7126
Epoch 10

Epoch 78/300
2191/2191 [==============================] - 0s 101us/step - loss: 0.5333 - val_loss: 0.6531
Epoch 79/300
2191/2191 [==============================] - 0s 101us/step - loss: 0.5325 - val_loss: 0.6561
Epoch 80/300
2191/2191 [==============================] - 0s 101us/step - loss: 0.5319 - val_loss: 0.6540
Epoch 81/300
2191/2191 [==============================] - 0s 102us/step - loss: 0.5309 - val_loss: 0.6563
Epoch 82/300
2191/2191 [==============================] - 0s 115us/step - loss: 0.5302 - val_loss: 0.6582
Epoch 83/300
2191/2191 [==============================] - 0s 103us/step - loss: 0.5293 - val_loss: 0.6599
Epoch 84/300
2191/2191 [==============================] - 0s 117us/step - loss: 0.5288 - val_loss: 0.6604
Epoch 85/300
2191/2191 [==============================] - 0s 117us/step - loss: 0.5288 - val_loss: 0.6639
Epoch 86/300
2191/2191 [==============================] - 0s 122us/step - loss: 0.5283 - val_loss: 0.6629
Epoch 87/300
2191/2191 [======================

Epoch 155/300
2191/2191 [==============================] - 0s 108us/step - loss: 0.5210 - val_loss: 0.6482
Epoch 156/300
2191/2191 [==============================] - 0s 103us/step - loss: 0.5207 - val_loss: 0.6493
Epoch 157/300
2191/2191 [==============================] - 0s 103us/step - loss: 0.5205 - val_loss: 0.6479
Epoch 158/300
2191/2191 [==============================] - 0s 109us/step - loss: 0.5203 - val_loss: 0.6509
Epoch 159/300
2191/2191 [==============================] - 0s 105us/step - loss: 0.5194 - val_loss: 0.6524
Epoch 160/300
2191/2191 [==============================] - 0s 103us/step - loss: 0.5186 - val_loss: 0.6501
Epoch 161/300
2191/2191 [==============================] - 0s 108us/step - loss: 0.5179 - val_loss: 0.6496
Epoch 162/300
2191/2191 [==============================] - 0s 108us/step - loss: 0.5173 - val_loss: 0.6505
Epoch 163/300
2191/2191 [==============================] - 0s 105us/step - loss: 0.5170 - val_loss: 0.6518
Epoch 164/300
2191/2191 [============

2191/2191 [==============================] - 0s 114us/step - loss: 0.5117 - val_loss: 0.6456
Epoch 232/300
2191/2191 [==============================] - 0s 103us/step - loss: 0.5118 - val_loss: 0.6494
Epoch 233/300
2191/2191 [==============================] - 0s 103us/step - loss: 0.5119 - val_loss: 0.6490
Epoch 234/300
2191/2191 [==============================] - 0s 103us/step - loss: 0.5117 - val_loss: 0.6454
Epoch 235/300
2191/2191 [==============================] - 0s 104us/step - loss: 0.5115 - val_loss: 0.6442
Epoch 236/300
2191/2191 [==============================] - 0s 104us/step - loss: 0.5118 - val_loss: 0.6482
Epoch 237/300
2191/2191 [==============================] - 0s 104us/step - loss: 0.5117 - val_loss: 0.6458
Epoch 238/300
2191/2191 [==============================] - 0s 103us/step - loss: 0.5116 - val_loss: 0.6440
Epoch 239/300
2191/2191 [==============================] - 0s 105us/step - loss: 0.5119 - val_loss: 0.6436
Epoch 240/300
2191/2191 [==========================

#### 5. Predict signals

In [12]:
encoded_img = encoder.predict(X_test_scaled)

In [13]:
columns = [f'glassnode_f{k}' for k in range(1,encoded_img.shape[1]+1)]
df_signals = pd.DataFrame(encoded_img, index=df_test.index, columns=columns)
df_signals.head(3)

,glassnode_f1,glassnode_f2,glassnode_f3,glassnode_f4,glassnode_f5
date,,,,,
2020-01-01,5.158278,12.852180,18.814423,0.0,10.379363
2020-01-02,4.300452,10.774682,13.848403,0.0,14.168247
2020-01-03,4.051940,9.483096,11.684012,0.0,15.455029


### Save signals

In [14]:
filepath = Path('../csv/signals_glassnode.csv')

In [15]:
df_signals.to_csv(filepath)